**注意** :由于pytorch版本的更新，现在 VGG16 所需的参数已经和以前不同，新版使用 weights 参数来代替旧版的 pretrained=True/False，所以内容以本案例为主即可。

本节讲解现有网络模型的使用与修改   
以 torchvision 中的 VGG16 为例   
torchvision.models.vgg16
### 参数1   
&emsp;weights: Optional[torchvision.models.VGG16_Weights] = None   控制是否加载预训练权重，以及加载哪一组权重，常见设置参数如下图：     
![VGG16_Weight](./images/VGG16_Weight.png)   
**代码示例** :
```python
from torchvision.models import vgg16, VGG16_Weights

# 加载预训练模型
model = vgg16(weights=VGG16_Weights.DEFAULT)

# 不加载预训练权重
model = vgg16(weights=None)
```
**注意** :   
旧版本中的 pretrained=True 已不推荐使用，等价于：
```python
model = vgg16(pretrained=True)
```
### 参数2
&emsp;progress: bool = True   当你第一次使用 weights=... 下载预训练模型时，是否在控制台显示下载进度条。   
### 参数3
&emsp;**kwargs   将其他关键字参数传给 torchvision.models.vgg.VGG 类的构造函数，用于更底层的定制，常见设置参数如下图：
![VGG16_kwargs](./images/VGG16_kwargs.png) 
**代码示例** :
```python
# 这种方法会替换最后一层全连接输出层为 Linear(4096, 10)，常用于迁移学习场景（自定义数据集）
model = vgg16(weights=None, num_classes=10)
```

关于ImageNet数据集   
ImageNet数据集需要去[ImageNet官网](https://www.image-net.org/download-images.php)下载，有问题的可以看这篇博客：[ImageNet数据集简介与下载详细步骤](https://blog.csdn.net/qq_36665989/article/details/119947229)   
官网中说明需要的三个文件：
Before using this class, it is required to download ImageNet 2012 dataset from here and place the files ILSVRC2012_devkit_t12.tar.gz and ILSVRC2012_img_train.tar or ILSVRC2012_img_val.tar based on split in the root directory.   
作用分别如下：   
![ImageNet_Files](./images/ImageNet_files.png)   
**注意** ：不需要解压就可以使用   
使用方法代码参考如下：   
```python
imagenet_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
root = "/path/to/ImageNet"
train_set = ImageNet(root=root, split='train', transform=imagenet_transform)
val_set = ImageNet(root=root, split='val', transform=imagenet_transform)
```
其中mean和std推荐使用代码示例中的值，因为这六个值都是根据 ImageNet 2012 数据集上统计得到的 RGB 通道的像素均值和标准差。

In [1]:
import torch
import torchvision
from torch import nn
from torchvision import models
from torchvision import transforms
from torch.utils.tensorboard import SummaryWriter

In [2]:
# 虽然土堆换了数据集，不再使用 ImageNet
# 但是这里还是给出确定可以运行的代码，我在我的电脑上跑通了，tensorboard 中也正确加载出了图片，需要使用 ImageNet 小伙伴可以参考

# imagenet_transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225]
#     )
# ])
# val_train = torchvision.datasets.ImageNet("../ImageNet/",
#                                           split = "val", transform = imagenet_transform)
# dataloader = torch.utils.data.DataLoader(val_train, batch_size = 2)
# step = 0
# writer = SummaryWriter("../logs/15_Model_Pretrained")
# for data in dataloader:
#     if step > 10:
#         break
#     imgs, targets = data
#     writer.add_images("Image_Net", imgs, step)
#     step += 1
# writer.close()

In [3]:
vgg16_true = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
vgg16_false = models.vgg16(weights=None)

In [4]:
# 可以通过下面这条命令来查看模型的权重文件存放位置
print(torch.hub.get_dir())

/Users/cliffkai/Code/model_cache/hub


In [5]:
print(vgg16_true)
train_data = torchvision.datasets.CIFAR10("../datasets/CIFAR10/", train = True, transform = torchvision.transforms.ToTensor(), download = True)
# VGG16 是分1000类，而 CIFAR10 只有是个类别，有两种法：
# 1.在模型后面添加一层，输入为1000，输出为10
# 2.将模型的最后输出修改为10

# 两种方法选择一种即可

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [6]:
# 方法1:
# 加到模型中
vgg16_true.classifier.add_module('add_linear', nn.Linear(1000, 100))
# 加到模型最后面
vgg16_true.add_module('add_linear', nn.Linear(100, 10))

print(vgg16_true)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [7]:
# 方法2:
vgg16_false.classifier[6] = nn.Linear(4096, 10)
print(vgg16_false)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1